In [1]:
import numpy as np


In [2]:
with open('./np_output/movie-xids.npy', 'rb') as f:
    Xids = np.load(f, allow_pickle=True)
with open('./np_output/movie-labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle=True)
with open('./np_output/movie-xmasks.npy', 'rb') as f:
    Xmasks = np.load(f, allow_pickle=True)

In [3]:
labels.shape

(156060, 5)

In [4]:
Xmasks

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [30]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((Xids,Xmasks,labels))


In [31]:
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.int64, name=None), TensorSpec(shape=(512,), dtype=tf.int64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [32]:
def map_fun(input_ids, mask, labels):
    return {'input_ids':input_ids,
            'attention_mask':mask},labels

In [33]:
dataset = dataset.map(map_fun)
dataset

<_MapDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [34]:
batch_size = 16

In [35]:
dataset = dataset.shuffle(10000).batch(batch_size,drop_remainder=True)
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [36]:
split = 0.9

In [37]:
size = int((Xids.shape[0]/batch_size)*split)
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

tf.data.experimental.save(train_ds, 'train')
tf.data.experimental.save(val_ds, 'val')

In [38]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [39]:
val_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [40]:
ds = tf.data.experimental.load('train', element_spec=train_ds.element_spec)
ds.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

: 